# Creating Animations with SymPy and Matplotlib

[back to SymPy page](index.ipynb)

[General information about Matplotlib animations](../plotting/matplotlib-animation.ipynb)

By default, SymPy uses Matplotlib for plotting,
but it is not quite straightforward to create
Matplotlib animations from SymPy plots.

This notebook shows a few ways how to get those animations running.

First, let's import some stuff from Matplotlib:

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

... and some stuff from SymPy:

In [ ]:
import sympy as sp
from sympy.plotting.plot import MatplotlibBackend

In [ ]:
x = sp.symbols('x')

Here we want to use JavaScript-based animations, for other options see
[the Matplotlib animation notebook](../plotting/matplotlib-animation.ipynb).

In [ ]:
plt.rcParams['animation.html'] = 'jshtml'

## Changing the Plotted Expression

One way for creating an animation is to create a single
[Plot](https://docs.sympy.org/latest/modules/plotting.html#sympy.plotting.plot.Plot) object
and later change the associated SymPy expression for each frame.

When we create the `Plot` object,
we just use a dummy expression,
but we should already select the desired range.

Note that we only want to get the `Plot` object,
we don't want to actually show the plot yet.

In [ ]:
p = sp.plot(0, (x, 0, 2 * sp.pi), show=False)

Normally, SymPy takes care of the plotting backend automatically,
but here we want to explicitly control the backend.

In [ ]:
backend = MatplotlibBackend(p)
plt.close(backend.fig)  # Avoid showing empty plot here

In the animation function, we simply have to switch the expression.

The tricky part is that we have to manually trigger the backend
in order to create the plot for the current frame.
And we have to make sure the previous content of the plot is erased.

In [ ]:
def func(frame):
    backend.ax.clear()
    p[0].expr = sp.sin(x - (frame / 5))
    # If there are multiple plots, also change p[1], p[2] etc.
    backend.process_series()

The rest is simple, we just have to use the `Figure` object from the SymPy backend
to create the animation.

In [ ]:
ani = FuncAnimation(backend.fig, func, frames=10)

In [ ]:
ani

## Changing the Whole Plot

If changing the expression isn't enough
(e.g. if you want to change the range of a plot),
you can use a slightly different approach.

In [ ]:
p = sp.plot(0, show=False)
backend = MatplotlibBackend(p)
plt.close(backend.fig)  # Avoid showing empty plot here

In [ ]:
def func(frame):
    backend.parent = sp.plot(sp.sin(x), (x, 0, 2 * sp.pi * (1 + frame/10)), show=False)
    backend.process_series()

In [ ]:
ani = FuncAnimation(backend.fig, func, frames=10)

In [ ]:
ani

## Plotting into an Existing Figure/Axes Object

SymPy normally creates `Figure` and `Axes` objects on its own.

To make it plot into an existing `Axes` object,
we can use a trick from https://stackoverflow.com/a/46813804.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2)
ax1.plot([4, 9, 7, 20, 6, 33, 13, 23, 16, 62, 8]);
ax2.spines['top'].set_color('none')
ax2.spines['right'].set_color('none')
ax2.spines['bottom'].set_position('zero')

In [ ]:
def func(frame):
    ax2.clear()
    p = sp.plot(sp.sin(x - (frame / 5)), (x, 0, 2 * sp.pi), show=False)
    backend = MatplotlibBackend(p)
    backend.ax = ax2
    backend.process_series()
    plt.close(backend.fig)

In [ ]:
ani = FuncAnimation(fig, func, frames=10)

In [ ]:
ani